# Image upscaler

- Uses [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for AI image upscaling
- Based on the colab demo available on the Real-ESRGAN repository

## Creating user input UI

- URLs for images go in the textbox below
- One URL per line

In [ ]:
from enum import Enum

import ipywidgets as wd

class Models(Enum):
  GENERAL = "realesr-general-x4v3"
  REALx4 = "RealESRGAN_x4plus"
  REALx2 = "RealESRGAN_x2plus"
  OVERSMOOTH = "RealESRNet_x4plus"
  OFFICIAL = "official ESRGAN_x4"
  ANIME = "RealESRGAN_x4plus_anime_6B"
  ANIMATION = "realesr-animevideov3"

dd_model = wd.Dropdown(
    description="Model: ",
    options = [model.value for model in Models],
    index = 0,
)
ft_denoise = wd.FloatText(
    value=0,
    description='Denoise:',
)
cb_face_enhance = wd.Checkbox(
    description="Face-enhancement",
    value = False,
    disabled = True,
)
cb_ceiling = wd.Checkbox(
    description="Use ceiling for outscale ratio",
    value = False,
)
it_width = wd.IntText(
    value=1920,
    description='Width:',
)
it_height = wd.IntText(
    value=1080,
    description='Height:',
)
tb_scale = wd.ToggleButton(
    value=False,
    description="Use scale",
    tooltip="Use a forced scale instead of trying to reach the target resolution",
)
ft_scale = wd.FloatText(
    value=2,
    description='Scale:',
    disabled = True,
)
ta_urls = wd.Textarea(
    placeholder="Type URLs for images",
    description="URLs:",
)

def validate_denoise(current_model):
  if current_model.new != Models.GENERAL.value:
    ft_denoise.disabled = True
  else:
    ft_denoise.disabled = False

def validate_face_enhance(current_model):
  if current_model.new == Models.GENERAL.value:
    cb_face_enhance.value = False
    cb_face_enhance.disabled = True
  else:
    cb_face_enhance.disabled = False

def validate_args(current_model):
  validate_denoise(current_model)
  validate_face_enhance(current_model)

def validate_scale(scale_is_forced):
  scale_is_forced = scale_is_forced.new
  it_width.disabled = scale_is_forced
  it_height.disabled = scale_is_forced
  ft_scale.disabled = not scale_is_forced

dd_model.observe(validate_args, names="value")
tb_scale.observe(validate_scale, names="value")

display(
    ta_urls,
    dd_model,
    ft_denoise,
    cb_face_enhance,
    cb_ceiling,
    wd.Label(value = "Resolution:"),
    it_width,
    it_height,
    tb_scale,
    ft_scale,
)

## Retrieving images

If no URLs are passed to the textbox on the UI above, an upload prompt is show to get images from the user's machine

In [ ]:
from google.colab import files
import os
import shutil

links = ta_urls.value.splitlines()

upload_folder = '/content/upload'
result_folder = '/content/results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

if len(links) == 0:
  # upload images
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
    if filename.endswith(".zip"):
      print(f"Decompressing {filename}")
      os.system(f"unzip -j {dst_path} -d {upload_folder}")
      os.remove(dst_path)
else:
  # Download image links if set
  for link in links:
   !wget --directory-prefix {upload_folder} {link}

## Setting up the environment

In [ ]:
!cd /content
!rm -rf Real-ESRGAN
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git

In [ ]:
%%capture
# Set up the environment
#!pip install facexlib
#!pip install gfpgan
!pip install basicsr
!pip install -r requirements.txt
!pip install torchvision==0.16.2
!pip install "numpy<2" # Downgrade numpy because of dependencies
!python setup.py develop

## Upscaling images

In [ ]:
%cd /content/Real-ESRGAN

from PIL import Image
import math

script_path = './inference_realesrgan.py'
dir_path = upload_folder
image_count = len(os.listdir(dir_path))
scale = ft_scale.value # used when target_res is None

target_res = (
     {"width": it_width.value, "height": it_height.value}
     if not tb_scale.value
     else None
)

args = [
    f"-n {dd_model.value}",
    # Denoise only works with the general model
    (f"-dn {ft_denoise.value}" if dd_model.value == Models.GENERAL.value else ""),
    ("--face_enhance" if cb_face_enhance.value else ""),
]
args = " ".join(args)

# If a target resolution has been set, run the upscaler individually on each
# image to try and reach the target
if target_res is not None:
  # Get all images in upload dir
  paths = [
      os.path.join(dir_path, file)
      for file in os.listdir(dir_path)
      if os.path.isfile(os.path.join(dir_path, file))
  ]

  for path in paths:
    with Image.open(path) as image:
        width, height = image.size

        w_ratio = target_res["width"]/width
        h_ratio = target_res["height"]/height

        scale = (max(w_ratio, h_ratio) if not cb_ceiling else math.ceil(max(w_ratio, h_ratio)))

        !python {script_path} {args} -i {path} --outscale {scale}
else:
  !python {script_path} {args} -i {upload_folder} -o {result_folder} --outscale {scale}

## Displaying results

In [ ]:
# utils for visualization
import cv2
import matplotlib.pyplot as plt
def display_img(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('Real-ESRGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_list = sorted(glob.glob(os.path.join(upload_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display_img(img_input, img_output)

## Downloading results

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} {result_folder}/*")
files.download(zip_filename)